In [7]:
import scipy.io as sio
import numpy as np 
import sys 

#145 145 220
def main():
    # See https://rslab.ut.ac.ir/data for information about this dataset (Urban)
    hs_image = sio.loadmat("data/Urban_R162.mat")
    # print(type(hs_image))

    # for key in hs_image:
    #     print(key)

    # Channels that are kept 
    # print(hs_image["SlectBands"])

    # Size of image 
    # print(hs_image["nRow"])
    # print(hs_image["nCol"])

    # Number of wavelengths 
    # print(hs_image["nBand"])

    # Matrix itself 
    # print(hs_image["Y"])

    # Not sure what this mean 
    # print(hs_image["maxValue"])

    
    matrix = np.array(hs_image["Y"])

    # Each row correspond to a channel/wavelength and the image is vectorized as a row vector (I am not sure how it is being flatten) 
    # I am not sure what the value in the matrix mean. They range from 0 to 1000. I am guessing it is how much is captured by a particular channel after normalization  
    print(matrix.shape)
    print(np.unique(matrix))

main()

(162, 94249)
[   0    1    2    3    4    5    6    7    8    9   10   11   12   13
   14   15   16   17   18   19   20   21   22   23   24   25   26   27
   28   29   30   31   32   33   34   35   36   37   38   39   40   41
   42   43   44   45   46   47   48   49   50   51   52   53   54   55
   56   57   58   59   60   61   62   63   64   65   66   67   68   69
   70   71   72   73   74   75   76   77   78   79   80   81   82   83
   84   85   86   87   88   89   90   91   92   93   94   95   96   97
   98   99  100  101  102  103  104  105  106  107  108  109  110  111
  112  113  114  115  116  117  118  119  120  121  122  123  124  125
  126  127  128  129  130  131  132  133  134  135  136  137  138  139
  140  141  142  143  144  145  146  147  148  149  150  151  152  153
  154  155  156  157  158  159  160  161  162  163  164  165  166  167
  168  169  170  171  172  173  174  175  176  177  178  179  180  181
  182  183  184  185  186  187  188  189  190  191  192  193  19